#### 1. Does reducing the batch_size (for instance, to 1) affect the reading performance?

It does! We get the following runtimes based on 10 runs each.
```python
Batch Size:    1 has a runtime of 7.12 ± 0.08 seconds
Batch Size:    4 has a runtime of 3.35 ± 0.06 seconds
Batch Size:   16 has a runtime of 2.42 ± 0.07 seconds
Batch Size:   64 has a runtime of 2.16 ± 0.03 seconds
Batch Size:  256 has a runtime of 2.10 ± 0.03 seconds
Batch Size: 1024 has a runtime of 2.11 ± 0.04 seconds
```

## Profiling and Evaluation Code

### Imports and Constants

In [1]:
import time

import torch
import torchvision
import ujson as json
from torch.utils.data import DataLoader
from torchvision import transforms
import pandas as pd

NUM_ITERATIONS = 10
BATCH_SIZES: list[int] = [1, 4, 16, 64, 256, 1024]

### Profiling Code

In [2]:
def run_profiling_code():
    resize: tuple[int, int] = (32, 32)
    trans = transforms.Compose([transforms.Resize(resize), transforms.ToTensor()])

    data_train: torch.Tensor = torchvision.datasets.FashionMNIST(
        root="./data/", train=True, transform=trans, download=True
    ).data

    all_times: dict[int, list[float]] = {}
    for i, batch_size in enumerate(BATCH_SIZES):
        print(f"{i+1}. batch size out of {len(BATCH_SIZES)}: {batch_size}.")
        data_loader = DataLoader(
            data_train, batch_size=batch_size, shuffle=True, num_workers=4
        )

        times: list[float] = []
        for _ in range(NUM_ITERATIONS):
            t0 = time.perf_counter()
            for _ in data_loader:
                pass
            t1: float = time.perf_counter()
            times.append(t1 - t0)
            print(f"\t\t{t1-t0:.2f} seconds")
        all_times[batch_size] = times

    filepath = "batch_size_speed_comparisons.json"
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(all_times, f)

### Result Printing

In [3]:
def print_results():
    data = torch.tensor(pd.read_json("batch_size_speed_comparisons.json").values)
    means, stds = data.mean(axis = 0), data.std(axis = 0)
    for batch_size, mean, std in zip(BATCH_SIZES, means, stds):
        print(f"Batch Size: {batch_size:4} has a runtime of {mean:4.2f} ± {std:.2f} seconds")